In [80]:
import numpy as np
import pandas as pd
import math
from numpy import random
from power_grid_model import LoadGenType, ComponentType, DatasetType, ComponentAttributeFilterOptions
from power_grid_model import PowerGridModel, CalculationMethod, CalculationType
from power_grid_model import initialize_array, attribute_dtype
from power_grid_model import FaultType, FaultPhase,ShortCircuitVoltageScaling

In [81]:
# 定义 CSV 文件名
csv_file = 'shortcircuit_more.csv'
with open(csv_file, 'w') as f:
    f.write('fault_node,node,ua,ua_angle,ub,ub_angle,uc,uc_angle,ia,ia_angle,ib,ib_angle,ic,ic_angle\n')  # 写入表头

In [82]:
#超参数
epoches = 1000

In [83]:
#生成随机数的归一化
def generate_random_sum(n, target_sum, min_val, max_val):
    # 生成 n 个随机数
    random_values = [random.uniform(min_val, max_val) for _ in range(n)]
    # 归一化并乘以目标总和
    total = sum(random_values)
    normalized_values = [value / total * target_sum for value in random_values]
    return normalized_values

In [84]:
def Y_calculation(line_data):
    """
    计算 导纳矩阵
    """
    # 初始化33x33的零矩阵
    Y = np.zeros((33, 33), dtype=complex)
    # 计算自导纳和互导纳
    for i in range(len(line)):
        from_node = int(line[i]['from_node'])
        to_node = int(line[i]['to_node'])
        r = line[i]['r1']
        x = line[i]['x1']
        
        # 计算自导纳
        y_ii = 1 / (r + 1j * x)
        
        # 添加自导纳
        Y[from_node, from_node] += y_ii
        Y[to_node, to_node] += y_ii
        
        # 添加互导纳
        Y[from_node, to_node] -= y_ii
        Y[to_node, from_node] -= y_ii
        
    return Y

In [85]:
def I_calculation(Y_maxtri,V_maxtri):
    V = pd.DataFrame(V_maxtri)
    V_np = V.to_numpy()#转换为numpy
    I=np.matmul(Y_maxtri,V_np)#节点电流矩阵
    return I

In [86]:
def transform_abs_angle(I_abi):
    # 计算模值
    magnitudes = np.abs(I_abi)
    
    # 计算角度（默认以弧度为单位）
    angles = np.angle(I_abi)
    
    # 将模值和角度组合成一个新的二维数组
    I_abs_angle = np.column_stack((magnitudes, angles))
    I_abs_angle = np.around(I_abs_angle,4)
    return I_abs_angle

In [87]:

#节点
node = initialize_array(DatasetType.input, ComponentType.node, 33)
node['id'] = np.arange(33)#节点编号
node['u_rated'] = 10.5e3#额定电压

#线路
line = initialize_array(DatasetType.input, ComponentType.line, 40)
line['id'] = np.arange(33,73)#线路编号
df_bus = pd.read_csv('Bus.csv')#读取节点文件
from_node = df_bus.iloc[:, 0]#提取第一列
to_node = df_bus.iloc[:, 1]#提取第二列
#转换为numpy
from_node = from_node.to_numpy()
to_node = to_node.to_numpy()
#赋值
line['from_node'] = from_node#线路起始节点
line['to_node'] = to_node#线路终止节点
line['c1'] = 0
line['tan1'] = 0
line['c0'] = 0
line['tan0'] = 0
line['from_status'] = 1
line['to_status'] = 1
line['i_n'] = 400
#负载
load = initialize_array(DatasetType.input, ComponentType.sym_load, 33)
load['id'] = np.arange(73, 106)#节点编号
df_load = pd.read_csv('loads.csv')#读取文件
load_node = df_load.iloc[:, 0]#提取第一列
load_node = load_node.to_numpy()
load['node'] = load_node
load['status'] = 1
load['type'] = LoadGenType.const_power
#电源
source = initialize_array(DatasetType.input, ComponentType.source, 1)
source['id'] = 106
source['node'] = 0
source['status'] = 1
source['u_ref'] = 1.2057142857#电压参考值

#故障（单相接地）
fault = initialize_array(DatasetType.input, ComponentType.fault, 1)
fault["id"] = [107]
fault["status"] = [1]

fault["fault_type"] = [FaultType.single_phase_to_ground]
fault["fault_phase"] = [FaultPhase.a]
fault["r_f"] = [15]
fault["x_f"] = [0.1]

In [88]:
for epoch in range(epoches):
    # 随机生成线路参数
    r1_line = [random.uniform(0.1, 0.5) for _ in range(40)]
    x1_line = [random.uniform(0.5, 1.5) for _ in range(40)]
    r0_line = [random.uniform(0.08, 1.1) for _ in range(40)]
    x0_line = [random.uniform(0.5, 2.0) for _ in range(40)]
    r1_line_series = pd.Series(r1_line)
    x1_line_series = pd.Series(x1_line)
    r0_line_series = pd.Series(r0_line)
    x0_line_series = pd.Series(x0_line)
    r1_line_np = r1_line_series.to_numpy()
    x1_line_np = x1_line_series.to_numpy()
    r0_line_np = r0_line_series.to_numpy()
    x0_line_np = x0_line_series.to_numpy()
    line['r1'] = r1_line_np
    line['x1'] = x1_line_np
    line['r0'] = r0_line_np
    line['x0'] = x0_line_np
    
    # 随机生成负载参数
    load_p = [3.755]
    load_p.extend(generate_random_sum(32, 3.755, 0.08, 1.1))
    load_q = [2.5]
    load_q.extend(generate_random_sum(32, 2.5, 0.05, 1.0))
    load_p = pd.Series(load_p)
    load_q = pd.Series(load_q)
    load_p = load_p.to_numpy()*1e6
    load_q = load_q.to_numpy()*1e6
    load['p_specified'] = load_p
    load['q_specified'] = load_q
    
    #随机生成故障点
    fault["fault_object"] = [random.randint(0, 33)]
    
    # all
    input_data = {
        ComponentType.node: node,
        ComponentType.line: line,
        ComponentType.sym_load: load,
        ComponentType.source: source,
        ComponentType.fault: fault,
    }
    # 验证输入数据
    from power_grid_model.validation import assert_valid_input_data
    assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.short_circuit)
    # 创建模型
    model = PowerGridModel(input_data)
    # 进行短路计算
    sc_res = model.calculate_short_circuit()
    #写入数据进CSV
    fault_node_csv = fault["fault_object"]
    fault_node_csv33 = [fault_node_csv[0]] * 33
    node_csv = sc_res[ComponentType.node]["id"]
    u_sc = sc_res[ComponentType.node]["u"]
    u_sc = np.around(u_sc,4)
    ua = [row[0] for row in u_sc]
    ub = [row[1] for row in u_sc]
    uc = [row[2] for row in u_sc]
    u_angle_sc = sc_res[ComponentType.node]["u_angle"]
    u_angle_sc33 = np.around(u_angle_sc,4)
    ua_angle = [row[0] for row in u_angle_sc33]
    ub_angle = [row[1] for row in u_angle_sc33]
    uc_angle = [row[2] for row in u_angle_sc33]
    #计算导纳矩阵,节点电流矩阵
    Y = Y_calculation(input_data[ComponentType.line])
    Ia_j = I_calculation(Y,ua)
    Ib_j = I_calculation(Y,ub)
    Ic_j = I_calculation(Y,uc)
    Ia = transform_abs_angle(Ia_j)
    Ib = transform_abs_angle(Ib_j)
    Ic = transform_abs_angle(Ic_j)
    ia = [row[0] for row in Ia]
    ib = [row[0] for row in Ib]
    ic = [row[0] for row in Ic]
    ia_angle = [row[1] for row in Ia]
    ib_angle = [row[1] for row in Ib]
    ic_angle = [row[1] for row in Ic]
    
    sc_res2 = pd.DataFrame({
            'fault_node': fault_node_csv33,
            'node': node_csv,
            'ua': ua,
            'ua_angle': ua_angle,
            'ub': ub,
            'ub_angle': ub_angle,
            'uc': uc,
            'uc_angle': uc_angle,
            'ia': ia,
            'ia_angle': ia_angle,
            'ib': ib,
            'ib_angle': ib_angle,
            'ic': ic,
            'ic_angle': ic_angle
    
        })
    sc_res2.to_csv(csv_file, mode='a', header=False, index=False)
    print("短路计算生成数据：",epoch+1)
print("数据生成完成")

    

短路计算生成数据： 1
短路计算生成数据： 2
短路计算生成数据： 3
短路计算生成数据： 4
短路计算生成数据： 5
短路计算生成数据： 6
短路计算生成数据： 7
短路计算生成数据： 8
短路计算生成数据： 9
短路计算生成数据： 10
短路计算生成数据： 11
短路计算生成数据： 12
短路计算生成数据： 13
短路计算生成数据： 14
短路计算生成数据： 15
短路计算生成数据： 16
短路计算生成数据： 17
短路计算生成数据： 18
短路计算生成数据： 19
短路计算生成数据： 20
短路计算生成数据： 21
短路计算生成数据： 22
短路计算生成数据： 23
短路计算生成数据： 24
短路计算生成数据： 25
短路计算生成数据： 26
短路计算生成数据： 27
短路计算生成数据： 28
短路计算生成数据： 29
短路计算生成数据： 30
短路计算生成数据： 31
短路计算生成数据： 32
短路计算生成数据： 33
短路计算生成数据： 34
短路计算生成数据： 35
短路计算生成数据： 36
短路计算生成数据： 37
短路计算生成数据： 38
短路计算生成数据： 39
短路计算生成数据： 40
短路计算生成数据： 41
短路计算生成数据： 42
短路计算生成数据： 43
短路计算生成数据： 44
短路计算生成数据： 45
短路计算生成数据： 46
短路计算生成数据： 47
短路计算生成数据： 48
短路计算生成数据： 49
短路计算生成数据： 50
短路计算生成数据： 51
短路计算生成数据： 52
短路计算生成数据： 53
短路计算生成数据： 54
短路计算生成数据： 55
短路计算生成数据： 56
短路计算生成数据： 57
短路计算生成数据： 58
短路计算生成数据： 59
短路计算生成数据： 60
短路计算生成数据： 61
短路计算生成数据： 62
短路计算生成数据： 63
短路计算生成数据： 64
短路计算生成数据： 65
短路计算生成数据： 66
短路计算生成数据： 67
短路计算生成数据： 68
短路计算生成数据： 69
短路计算生成数据： 70
短路计算生成数据： 71
短路计算生成数据： 72
短路计算生成数据： 73
短路计算生成数据： 74
短路计算生成数据： 75
短路计算生成数据： 76
短路计算生成数据： 77
短路计算生成数据